### Import cw-simulate module and create new instance

In [ ]:
import { SimulateCosmWasmClient } from "@oraichain/cw-simulate";
import { SigningCosmWasmClient } from "@cosmjs/cosmwasm-stargate"
import { coins, GasPrice } from '@cosmjs/stargate';


if(simulate) {
  senderAddress = "orai14vcw5qk0tdvknpa38wz46js5g7vrvut8lk0lk6";
  client = new SimulateCosmWasmClient({
    chainId: "Oraichain",
    bech32Prefix: "orai",
    metering: true
  });
  client.app.bank.setBalance(senderAddress, coins('10000000', 'orai'));
} else {  
  await keplr.enable('Oraichain-testnet');
  const signer = await keplr.getOfflineSignerAuto('Oraichain-testnet');
  senderAddress = (await signer.getAccounts())[0].address;
  client = await SigningCosmWasmClient.connectWithSigner('https://testnet.rpc.orai.io', signer, {  
      gasPrice: GasPrice.fromString('0.002orai')  
  });
}

console.log(await client.getBalance(senderAddress, 'orai'));

### Deploy contract

In [ ]:
import { CwStarterClient } from "./contracts";

const wasmCode = new Uint8Array(await fetch("/wasm/cw-starter.wasm").then((res) => res.arrayBuffer()));
const { codeId } = await client.upload(senderAddress, wasmCode, "auto");
const { contractAddress } = await client.instantiate(
  senderAddress,
  codeId,
  { 
    admin: senderAddress   
  },
  "cw-starter",
  "auto"
);
console.log('contractAddress: ' + contractAddress);

cwStarter = new CwStarterClient(client, senderAddress, contractAddress);

### Test execute create poll valid

In [ ]:
res = await cwStarter.createPoll({
    pollId: 'some_id',
    question: "What's your favourite Cosmos coin?",
    options: ['Cosmos Hub', 'Juno', 'Osmosis']
});

console.log(res);

### Test execute create poll invalid

In [ ]:
res = await cwStarter.createPoll({
    pollId: 'some_id',
    question: "What's your favourite Cosmos coin?",
    options: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
});

console.log(res);

### Test execute vote valid

In [ ]:
// Create the vote, first time voting
res = await cwStarter.vote({
    pollId: 'some_id',
    vote: 'Juno'
});
  
  // Change the vote
  res = await cwStarter.vote({
    pollId: 'some_id',
    vote: 'Osmosis'
});

### Test execute vote invalid

In [ ]:
// Vote on a now existing poll but the option "DVPN" does not exist
res = await cwStarter.vote({
    pollId: 'some_id',
    vote: 'DVPN'
});

console.log(res)

### Test query all polls

In [ ]:
// Create a second poll
await cwStarter.createPoll({
    pollId: 'some_id_2',
    question: "What's your colour?",
    options: ['Red', 'Green', 'Blue']
});
  
// Query
res = await cwStarter.allPolls();
console.log(res);

### Test query poll

In [ ]:
// Query for the poll that exists
res = await cwStarter.poll({
    pollId: 'some_id'
});

console.log(res);

// Query for the poll that does not exists
res = await cwStarter.poll({
    pollId: 'some_id_not_exist'
});
  
// expect not exist
console.log(res)

### Test query vote

In [ ]:
// Query for a vote that exists
res = await cwStarter.getVote({
    pollId: 'some_id',
    address: senderAddress
});
  
// Expect the vote to exist
console.log(res);
  
// Query for a vote that does not exists
  
res = await cwStarter.getVote({
    pollId: 'some_id_not_exist',
    address: senderAddress
});

console.log(res)
// Expect the vote to not exist